In [ ]:
# проверяется гипотеза о наличии у пользователей предпочтений в покупках (склонность к покупкам товаров одного бренда)

In [23]:
import pandas as pd

In [13]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/383523/lesson_3_data__1_.csv', 
                 encoding='windows-1251',
                 usecols=[i for i in range(1,21)]
                )
df.head(1)

,tk,pl,dia,hs,tc,cta,id_art,id_subsubfam,id_subfam,id_fam,id_famn,id_seccion,id_subagr,id_agr,vta,uni,id_artn,art_sp,fam_sp,fam_en
0,120071109002055793,1,2007/11/09,0.505729,110000761,11000076,21895,101070640100,1010706401,10107064,10107064,10107,101,1,0.68,1.0,21895,MARAVILLA 500 G Store_Brand,PASTA ALIMENTICIA SE,pasta


In [16]:
user_df = df[['tc', 'art_sp']]
user_df = user_df.rename(columns={'tc': 'user_id', 'art_sp':'brand_info'})
user_df.head(2)

,user_id,brand_info
0,110000761,MARAVILLA 500 G Store_Brand
1,110000761,FIDEO CABELLIN 500 G Store_Brand


In [22]:
user_df['brand'] = user_df.brand_info.apply(lambda x: x.split(' ')[-1])
user_df.head(2)

,user_id,brand_info,brand
0,110000761,MARAVILLA 500 G Store_Brand,Store_Brand
1,110000761,FIDEO CABELLIN 500 G Store_Brand,Store_Brand
